In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
from define_helpers import select_rep_site, _define_cryptic_status, select_rep_prox_site, select_rep_five_end, construct_bed_name
import os


# approach

### Input last exons - selecting representative PAS/last exon:

le_ids can have multiple predictions, need to select a single one.
Strategy:
(starting with novel last exons used for quantification) 
1. Prefer events that match atlas
2. if multiple, select min atlas distance
3. If matches, s





## To-dos:
- fix updating to atlas for -ve strand matching pas
- run selection for all novel 3'UTR extensions
- Implement selection for annotated proximal sites of 3'UTR extensions (i.e. most distal annotated site)


# 

In [2]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-12-10_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")

# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")


In [3]:
print(quant_uniq_le.columns)
quant_uniq_le.subset(lambda df: ~df.transcript_id.str.startswith("ENST"))

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'transcript_id', 'exon_number',
       'region_rank', 'Start_ref', 'End_ref', 'transcript_id_ref',
       '3p_extension_length', 'event_type', 'ref_gene_id', 'ref_gene_name',
       'le_number', 'le_id'],
      dtype='object')


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24476667,.,+,.,PAPA.NT_19074713_S59.398,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
1,chr1,.,exon,24468985,24476667,.,+,.,PAPA.ctrl_ctrl_3.424,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
2,chr1,.,exon,24468985,24475256,.,+,.,PAPA.TDP43_19065409_S29.438,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
3,chr1,.,exon,24468985,24473773,.,+,.,PAPA.Cont-B_S2.334,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
4,chr1,.,exon,24468985,24474852,.,+,.,PAPA.DZ_curves_1_3.360,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36889,chrY,.,exon,13234576,13234826,.,-,.,PAPA.NT_19074729_S12.25635,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36890,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43-F_S6.20683,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36891,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43_19065413_S19.26168,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36892,chrY,.,exon,13234583,13234826,.,-,.,PAPA.DZ_curves_0_2.21889,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7


In [4]:
dexseq_df.columns

Index(['experiment_name', 'binID', 'groupID', 'featureID', 'exonBaseMean',
       'dispersion', 'stat', 'pvalue', 'padj', 'UsageCoefficient_base',
       'UsageCoefficient_contrast', 'log2fold_contrast_base', 'le_id',
       'gene.qvalue', 'contrast_name', 'gene_id', 'gene_name', 'event_type',
       'annot_status', 'transcript_id', 'chromosome', 'strand', 'start', 'end',
       'mean_PPAU_base', 'mean_PPAU_treatment', 'delta_PPAU_treatment_control',
       'simple_event_type'],
      dtype='object')

In [5]:
novel_le.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU'],
      dtype='object')

In [6]:
# summary df of cryptic events following manual validation of bleedthrough events
cryptics_df = pd.read_csv("../../preprocessing/processed/2023-12-10_cryptics_summary_all_events_bleedthrough_manual_validation.tsv", sep="\t")

# load in IDs to remove entirely from analysis (remove as cryptic, but make sure nto included in overall background)
with open("../../preprocessing/processed/2023-12-10_cryptics_manual_validation_fail_le_ids.txt", "r") as infile:
    manual_validation_fail_ids = [line.rstrip("\n") for line in infile]

# get a dict of set of le_ids in each case
event_type_le_ids = cryptics_df.groupby("simple_event_type")["le_id"].agg(set).to_dict()
event_type_gene_names = cryptics_df.groupby("simple_event_type")["gene_name"].agg(set).to_dict()

for event_type, ids in event_type_le_ids.items():
    print(f"Event type - {event_type} - number of IDs - {len(ids)}")

print(f"Number of cryptic IDs failing manual validation - {len(manual_validation_fail_ids)}")

Event type - bleedthrough - number of IDs - 20
Event type - bleedthrough,spliced - number of IDs - 9
Event type - distal_3utr_extension - number of IDs - 86
Event type - proximal_3utr_extension - number of IDs - 20
Event type - spliced - number of IDs - 92
Number of cryptic IDs failing manual validation - 34


In [7]:
print(event_type_le_ids.keys())

dict_keys(['bleedthrough', 'bleedthrough,spliced', 'distal_3utr_extension', 'proximal_3utr_extension', 'spliced'])


In [8]:
# filter out failed IDs from dexseq df (so not included as background)
dexseq_df = dexseq_df[~dexseq_df["le_id"].isin(manual_validation_fail_ids)]
dexseq_df

,experiment_name,binID,groupID,featureID,exonBaseMean,dispersion,stat,pvalue,padj,UsageCoefficient_base,...,annot_status,transcript_id,chromosome,strand,start,end,mean_PPAU_base,mean_PPAU_treatment,delta_PPAU_treatment_control,simple_event_type
0,brown_i3_cortical,ENSG00000021645.20:E001,ENSG00000021645.20,E001,32.627088,0.044220,90.962787,1.463997e-21,2.250563e-19,1.452069,...,novel,PAPA.TDP-4.18197.2,chr14,+,7.829867e+07,7.830620e+07,0.001914,0.033848,0.031934,spliced
1,brown_i3_cortical,ENSG00000021645.20:E002,ENSG00000021645.20,E002,622.521901,0.022117,135.739619,2.274890e-31,5.061630e-29,12.675156,...,annotated,"ENST00000557594.5,ENST00000428277.6,ENST000002...",chr14,+,7.986104e+39,7.986248e+39,0.998086,0.934776,-0.063310,spliced
2,brown_i3_cortical,ENSG00000021645.20:E002,ENSG00000021645.20,E002,622.521901,0.022117,135.739619,2.274890e-31,5.061630e-29,12.675156,...,annotated,"ENST00000557594.5,ENST00000428277.6,ENST000002...",chr14,+,7.986104e+39,7.986248e+39,0.998086,0.934776,-0.063310,spliced
3,brown_i3_cortical,ENSG00000048649.14:E001,ENSG00000048649.14,E001,163.262346,0.037855,88.319669,5.568698e-21,8.407740e-19,4.515264,...,novel,"PAPA.doxconc_DOX_0075_2.11065.7,PAPA.TDP43-G_S...",chr11,-,7.781315e+71,7.781379e+71,0.090158,0.462779,0.372622,spliced
4,brown_i3_cortical,ENSG00000048649.14:E001,ENSG00000048649.14,E001,163.262346,0.037855,88.319669,5.568698e-21,8.407740e-19,4.515264,...,annotated,"PAPA.doxconc_DOX_0075_2.11065.7,PAPA.TDP43-G_S...",chr11,-,7.781315e+71,7.781379e+71,0.090158,0.462779,0.372622,spliced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109646,zanovello_skndz_curve_1,ENSG00000280789.2:E002,ENSG00000280789.2,E002,99.077638,12.000000,0.006587,9.353145e-01,9.785953e-01,5.982423,...,novel,PAPA.chx_tdp_CTRL_ctrl_4.15153.2,chr16,+,2.981955e+07,2.982252e+07,0.181220,0.333333,0.152113,distal_3utr_extension
109647,zanovello_skndz_curve_1,ENSG00000280832.2:E001,ENSG00000280832.2,E001,75.778902,0.099478,0.075875,7.829679e-01,9.189656e-01,6.014731,...,annotated,ENST00000626810.1,chr11,-,1.263502e+08,1.263518e+08,0.133144,0.147606,0.014462,spliced
109648,zanovello_skndz_curve_1,ENSG00000280832.2:E002,ENSG00000280832.2,E002,402.882205,0.085732,0.087526,7.673463e-01,9.134343e-01,12.145436,...,annotated,ENST00000629441.2,chr11,-,1.263409e+08,1.263423e+08,0.866856,0.852394,-0.014462,spliced
109649,zanovello_skndz_curve_1,ENSG00000282508.2:E001,ENSG00000282508.2,E001,43.093715,0.060733,0.017535,8.946533e-01,9.665471e-01,4.668928,...,annotated,ENST00000633109.1,chr19,-,2.112510e+05,2.117150e+05,0.678888,0.582547,-0.096341,spliced


In [9]:
# get le_ids that are novel 3'UTR ectensions
le_id_d3utr = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
len(le_id_d3utr)

3318

In [10]:
cryp_le_ids = event_type_le_ids["distal_3utr_extension"]
cryp_gene_names = event_type_gene_names["distal_3utr_extension"]

print(f"Number of cryptic 3'UTR extensions - {len(cryp_le_ids)}")
print(f"Number of cryptic 3'UTR extension-containing genes - {len(cryp_gene_names)}")

Number of cryptic 3'UTR extensions - 86
Number of cryptic 3'UTR extension-containing genes - 86


In [11]:
cryp_le_ids_prox = event_type_le_ids["proximal_3utr_extension"]
cryp_gene_names_prox = event_type_gene_names["proximal_3utr_extension"]

print(f"Number of cryptic 3'UTR extensions where proximal is cryptic - {len(cryp_le_ids_prox)}")
print(f"Number of cryptic 3'UTR extension-containing genes where proximal is cryptic - {len(cryp_gene_names_prox)}")

Number of cryptic 3'UTR extensions where proximal is cryptic - 20
Number of cryptic 3'UTR extension-containing genes where proximal is cryptic - 20


In [12]:
# add le_id used in downstream analysis
novel_le = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"]))

In [13]:
# Select representative PAS for distal 3'UTR extension events (novel)
novel_le_rep_d3utr, rep_choices_d3utr = select_rep_site(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_d3utr))), id_col="le_id_quant")


print(f"Number of novel 3'UTR extenstion intervals before selecting representative LEs - {len(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_d3utr))))}")
print(f"Number of novel 3'UTR extension intervals after selecting representative LEs - {len(novel_le_rep_d3utr)}")
print({dec: len(ids) for dec, ids in rep_choices_d3utr.items()})

Number of novel 3'UTR extenstion intervals before selecting representative LEs - 15660
Number of novel 3'UTR extension intervals after selecting representative LEs - 3347
{'atlas_1_pred': 1530, 'atlas_max_datasets': 531, 'atlas_max_datasets_shortest': 341, 'motif_1_min': 863, 'motif_shortest_min': 53}


In [14]:
# double check if any IDs dropped
le_id_d3utr - set(novel_le_rep_d3utr.le_id_quant)

set()

In [15]:
# now need to select representative proximal site for each distal site, since quantification approach lumps together all annotated proximal sites (and compares them to extension)
# since le_ids are annotated sequentially, know that immediately preceding le_number is the partner le_id
d3utr_le_id_split = novel_le_rep_d3utr.le_id_quant.str.split("_", regex=False, expand=True)
print(d3utr_le_id_split)
d3utr_le_id_split[1] = d3utr_le_id_split[1].astype(int).subtract(1)

# reconstruct le_id (corresponding to proximal le_id)
d3utr_prox_le_id = d3utr_le_id_split[0].str.cat(d3utr_le_id_split[1].astype(str), sep="_")
d3utr_prox_le_id

                       0  1
6     ENSG00000001461.17  6
85    ENSG00000023902.14  3
98    ENSG00000031698.13  3
102   ENSG00000033122.21  5
143    ENSG00000053372.5  3
...                  ... ..
1126  ENSG00000129824.16  2
1769   ENSG00000154620.6  2
2057  ENSG00000165246.15  3
60    ENSG00000012817.16  3
3033   ENSG00000230663.1  3

[3347 rows x 2 columns]


6       ENSG00000001461.17_5
85      ENSG00000023902.14_2
98      ENSG00000031698.13_2
102     ENSG00000033122.21_4
143      ENSG00000053372.5_2
                ...         
1126    ENSG00000129824.16_1
1769     ENSG00000154620.6_1
2057    ENSG00000165246.15_2
60      ENSG00000012817.16_2
3033     ENSG00000230663.1_2
Name: 0, Length: 3347, dtype: object

In [16]:
#subset for annotated last exons with 3'end proximal to 3'UTR extensions
quant_uniq_le_d3utr_prox = quant_uniq_le.subset(lambda df: df.le_id.isin(set(d3utr_prox_le_id.values)))
quant_uniq_le_d3utr_prox


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,150158818,150159404,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
3,chr1,.,exon,150159497,150160065,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
4,chr1,.,exon,150158818,150159404,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,133.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
12479,chrY,.,exon,19705416,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
12480,chrY,.,exon,19705419,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
12481,chrY,.,exon,19705424,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2


In [17]:
# double check if any cryptic genes dropped
# [le_id.split("_")] prevents trying to unpack each character in string
# list(le_id_d3utr)

# which d3utr_extension annotated le_ids do not have proximal le_id extracted for the event?
missing_ids_prox = set("_".join([i, str(int(n) - 1)]) for le_id in le_id_d3utr for i,n in [le_id.split("_")]) - set(quant_uniq_le_d3utr_prox.le_id)
# go back to original ids
missing_ids = set("_".join([i, str(int(n) + 1)]) for le_id in missing_ids_prox for i,n in [le_id.split("_")])

missing_ids


{'ENSG00000126214.22_3',
 'ENSG00000160439.16_4',
 'ENSG00000165156.15_3',
 'ENSG00000176222.9_3',
 'ENSG00000196268.12_4',
 'ENSG00000213923.13_4'}

In [18]:
# are any of these 6 cryptic?
print(cryptics_df[cryptics_df.le_id.isin(missing_ids)])


Empty DataFrame
Columns: [le_id, gene_name, simple_event_type, n_exper_cryptic, experiment_name, annot_status, chromosome, start, end, strand]
Index: []


In [19]:
# grab out gene names
missing_ids_gn = set(dexseq_df[dexseq_df.le_id.isin(missing_ids)].gene_name)
# look in input quant GTF to assess
quant_uniq_le.subset(lambda df: df.gene_name.isin(missing_ids_gn))[["gene_name", "gene_id", "le_id", "simple_event_type"]]

,Chromosome,Start,End,Strand,gene_name,gene_id,le_id
0,chr8,123248455,123250320,-,ZHX1,"ENSG00000259305.6,ENSG00000165156.15","ENSG00000259305.6,ENSG00000165156.15_1"
1,chr8,123249659,123250320,-,ZHX1,"ENSG00000259305.6,ENSG00000165156.15","ENSG00000259305.6,ENSG00000165156.15_1"
2,chr8,123250129,123250320,-,ZHX1,"ENSG00000259305.6,ENSG00000165156.15","ENSG00000259305.6,ENSG00000165156.15_1"
3,chr14,103654825,103654838,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_1"
4,chr14,103679545,103679798,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_2"
5,chr14,103685036,103685316,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_3"
6,chr14,103685637,103686200,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_4"
7,chr14,103685637,103685896,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_4"
8,chr14,103685637,103685702,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_4"
9,chr14,103693634,103694047,+,KLC1,"ENSG00000126214.22,ENSG00000256500.6","ENSG00000126214.22,ENSG00000256500.6_5"


In [20]:
# novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.str.contains(",", regex=False))
# novel_le.subset(lambda df: df.le_id_quant.str.contains(",", regex=False))[["le_id_quant", "gene_name_ref"]]

In [21]:
# quant_uniq_le.subset(lambda df: df.le_id.str.contains(",", regex=False)).subset(lambda df: df.le_id.str.contains("|".join(missing_ids)))[["le_id", "gene_name", "gene_id"]]

There are 6 background events dropped. I don't know why (and no time to understand why), but they are combo gene names which suggests complex loci. As 6 events and in background, I do not expect this to drastically affect the results, so I will leave open for now...

In [22]:
# select representative proximal site for partner annotated ALEs (most distal of annotated PAS)
sel_le_d3utr_prox = select_rep_prox_site(quant_uniq_le_d3utr_prox, id_col="le_id")
sel_le_d3utr_prox

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,150159497,150160065,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
3,chr1,.,exon,109237730,109238182,.,+,.,ENSG00000031698.13,SARS1,...,last,109237373,109237730,ENST00000468588.1,NULL,first_exon_spliced,ENSG00000031698.13,SARS1,2.0,ENSG00000031698.13_2
4,chr1,.,exon,70121779,70121994,.,+,.,ENSG00000033122.21,LRRC7,...,last,70107826,70121779,ENST00000415775.2,NULL,last_exon_spliced,ENSG00000033122.21,LRRC7,4.0,ENSG00000033122.21_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,chrY,.,exon,13705224,13706024,.,+,.,ENSG00000154620.6,TMSB4Y,...,last,13704435,13705224,ENST00000284856.4,NULL,first_exon_spliced,ENSG00000154620.6,TMSB4Y,1.0,ENSG00000154620.6_1
4318,chrY,.,exon,14840412,14845650,.,+,.,ENSG00000165246.15,NLGN4Y,...,last,"14830519,14830519,14830519,14830519","14840412,14840412,14840412,14840412","ENST00000339174.9,ENST00000382868.5,ENST000003...","NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000165246.15,NLGN4Y,2.0,ENSG00000165246.15_2
4319,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
4320,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2


In [23]:
# check no further cryptics/le_ids dropped
prox_missing_ids_prox = set("_".join([i, str(int(n) - 1)]) for le_id in le_id_d3utr for i,n in [le_id.split("_")]) - set(sel_le_d3utr_prox.le_id)
print(f"Numebr of missing IDs - {len(prox_missing_ids_prox)}")
# are they identical to previous IDs?
prox_missing_ids_prox.intersection(missing_ids_prox)

Numebr of missing IDs - 6


{'ENSG00000126214.22_2',
 'ENSG00000160439.16_3',
 'ENSG00000165156.15_2',
 'ENSG00000176222.9_2',
 'ENSG00000196268.12_3',
 'ENSG00000213923.13_3'}

In [24]:
# same 6 IDs. Don't think losing 6 events from background is going to make a dramatic difference...
# remove missing IDs from novel distal DF, so don't have inbalanced numbers of proximal/distal events
novel_le_rep_d3utr = novel_le_rep_d3utr.subset(lambda df: ~df.le_id_quant.isin(missing_ids))

In [25]:
# Alternative definition of background (more conservative)
# not significant in any SH-SY5Y dataset, but expressed/evaluated in at least one of them (so know is multi-ALE in at least one dataset)
# NB: extra-check that expressed/evaluated in all 3? Then confident that is a multi-ALE gene robustly expressed/detected
shsy5y_datasets = ["zanovello_shsy5y_curve_0075", "zanovello_shsy5y_chx_kd_ctl_vs_ctl_ctl", "brown_shsy5y"]
dexseq_df_shsy5y = dexseq_df.loc[dexseq_df.experiment_name.isin(shsy5y_datasets), :]

# first get an experiment count for each event
gene_n_shsy5y_expressed = (dexseq_df_shsy5y
.groupby("groupID")
["experiment_name"]
.nunique()
)

print("Gene counts for the number of SH-SY5Y datasets in which genes are expressed")
print(gene_n_shsy5y_expressed.value_counts())

# get genes expressed in all SH-SY5Y datasets but with no sig change in any of them 
ns_gene_ids_shsy5y = (dexseq_df_shsy5y.loc[dexseq_df_shsy5y.groupID.isin(set(gene_n_shsy5y_expressed[gene_n_shsy5y_expressed == len(shsy5y_datasets)].index)), :]
.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
               .groupby("groupID")
               ["reg_status"]
               .sum()
               .loc[lambda x: x == 0]
)

# set of
ns_le_ids_shsy5y = set(dexseq_df_shsy5y.loc[dexseq_df_shsy5y["groupID"].isin(ns_gene_ids_shsy5y.index), "le_id"])


print(f"Number of ns ALE-containing genes expressed in all SH-SY5Y datasets - {len(ns_gene_ids_shsy5y.index)}")
print(f"Number of ns ALE isoforms of genes expressed in all SH-SY5Y datasets - {len(set(ns_le_ids_shsy5y))}")

# le_ids in genes with no differential usage in any dataset that are also expressed/assessed in all datasets
ns_le_ids_shsy5y_all = set(dexseq_df_shsy5y[dexseq_df_shsy5y.le_id.isin(ns_le_ids_shsy5y)]
.groupby("le_id")
["experiment_name"]
.nunique()
.loc[lambda x: x == 3]
.index
)

print("Of the genes containing ALE isoforms, how many datasets are the individual isoforms expressed in?")
print(dexseq_df_shsy5y[dexseq_df_shsy5y.le_id.isin(ns_le_ids_shsy5y)]
.groupby("le_id")
["experiment_name"]
.nunique()
.value_counts()
)

print(f"Number of ns ALE isoforms expressed in all SH-SY5Y datasets - {len(ns_le_ids_shsy5y_all)}")
# (dexseq_df.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
#                .groupby("groupID")
#                ["reg_status"]
#                .sum()
#                
#              )


Gene counts for the number of SH-SY5Y datasets in which genes are expressed
experiment_name
3    2582
1    1068
2     704
Name: count, dtype: int64
Number of ns ALE-containing genes expressed in all SH-SY5Y datasets - 1577
Number of ns ALE isoforms of genes expressed in all SH-SY5Y datasets - 3624
Of the genes containing ALE isoforms, how many datasets are the individual isoforms expressed in?
experiment_name
3    3304
1     185
2     135
Name: count, dtype: int64
Number of ns ALE isoforms expressed in all SH-SY5Y datasets - 3304


In [26]:
# now want to combine the two with minimal info required for making maps
# 3'end coordinates, gene name, le_id, site type, regulation status

# define site_type columns
sel_le_d3utr_prox.site_type = "proximal"
novel_le_rep_d3utr.site_type = "distal"


In [27]:
# le_ids of novel extensions with no change in any experiment
ns_le_ids_d3utr = set(dexseq_df.loc[(dexseq_df["le_id"].isin(ns_le_ids_shsy5y_all.intersection(le_id_d3utr))), "le_id"].drop_duplicates())


# now get le_ids of partner proximal les
ns_le_ids_d3utr_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) - 1)]) for le_id in ns_le_ids_d3utr])
print(len(ns_le_ids_d3utr) == len(ns_le_ids_d3utr_prtnr))
ns_le_ids_d3utr_comb = ns_le_ids_d3utr.union(ns_le_ids_d3utr_prtnr)
len(ns_le_ids_d3utr_comb)

True


1636

In [28]:
# get proximal partner of cryptic les 
cryp_le_ids_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) - 1)]) for le_id in list(cryp_le_ids)])
# combination of cryptic and background le_ids
cryp_le_ids_comb = cryp_le_ids.union(cryp_le_ids_prtnr)

In [29]:
# get combined list of cryptic IDs
# here partner is the novel 3'UTR extension, so add 1 to le_id
cryp_le_ids_prox_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) + 1)]) for le_id in list(cryp_le_ids_prox)])
# combination of cryptic and background le_ids
cryp_le_ids_prox_comb = cryp_le_ids_prox.union(cryp_le_ids_prox_prtnr)

In [30]:
sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_comb, ns_le_ids_d3utr_comb, id_col="le_id"))
novel_le_rep_d3utr = novel_le_rep_d3utr.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_comb, ns_le_ids_d3utr_comb, id_col="le_id_quant"))

sel_le_prox_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_prox_comb, ns_le_ids_d3utr_comb, id_col="le_id"))
novel_le_rep_prox_d3utr = novel_le_rep_d3utr.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_prox_comb, ns_le_ids_d3utr_comb, id_col="le_id_quant"))

# sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))
# sel_novel_le_d3utr = sel_novel_le_d3utr.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id_ref.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))

In [31]:
print(novel_le_rep_d3utr.cryptic_status.value_counts())
print("\nNumber of duplicated IDs")
# how many IDs are duplicated with distinct regions?
novel_d3utr_dup_ids = set(novel_le_rep_d3utr.drop_duplicate_positions().as_df().le_id_quant.value_counts().loc[lambda x: x > 1].index)
# subset is for le_ids that have more than one entry. Then collapse to a single entry for each le_id and count the number of unique IDs
novel_le_rep_d3utr[novel_le_rep_d3utr.le_id_quant.isin(novel_d3utr_dup_ids)].drop_duplicate_positions().as_df().drop_duplicates(subset="le_id_quant").cryptic_status.value_counts()

cryptic_status
NULL          2447
background     808
cryptic         86
Name: count, dtype: int64

Number of duplicated IDs


cryptic_status
NULL          20
background     9
Name: count, dtype: int64

In [32]:
# Only 9 background events still have multiple entries after collapsing 3'ends

# number of events in selected proximal df - should be seeing 86 & 808
sel_le_d3utr_prox.cryptic_status.value_counts()

cryptic_status
NULL          3175
background    1019
cryptic        128
Name: count, dtype: int64

In [33]:
# how many events are duplicated when selecting a representative proximal PAS?
sel_d3utr_prox_dup_ids = set(sel_le_d3utr_prox.drop_duplicate_positions().le_id.value_counts().loc[lambda x: x > 1].index)
sel_le_d3utr_prox[sel_le_d3utr_prox.le_id.isin(sel_d3utr_prox_dup_ids)].drop_duplicate_positions().as_df().drop_duplicates(subset="le_id").cryptic_status.value_counts()

cryptic_status
NULL          23
background     7
cryptic        3
Name: count, dtype: int64

In [34]:
print("Number of proximal PAS regions with distinct 5'ends ")
print(sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

print("Number of proximal PAS regions with distinct 3'ends")
print(sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )


Number of proximal PAS regions with distinct 5'ends 
33
Number of proximal PAS regions with distinct 3'ends
0


In [35]:
print("Number of distal PAS regions with distinct 5'ends ")
print(novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

print("Number of distal PAS regions with distinct 3'ends")
print(novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

Number of distal PAS regions with distinct 5'ends 


0
Number of distal PAS regions with distinct 3'ends
28


Proximal regions - all have different 5'ends/exon starts
Distal regions - all have distinct 3'ends

In [36]:
# look at duplicated extensions/distal PAS
novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids)).drop_duplicate_positions()[["le_id_quant", "gene_name_ref", "Name", "cryptic_status"]]

,Chromosome,Start,End,Strand,le_id_quant,gene_name_ref,Name,cryptic_status
0,chr1,70439100,70445530,+,ENSG00000116761.12_2,CTH,1:70445533:+,background
1,chr1,70439100,70445536,+,ENSG00000116761.12_2,CTH,1:70445533:+,background
2,chr1,33494217,33501611,+,ENSG00000121903.15_4,ZSCAN20,1:33501609:+,background
3,chr1,33494217,33501607,+,ENSG00000121903.15_4,ZSCAN20,1:33501609:+,background
4,chr1,44778575,44779122,+,ENSG00000142937.12_2,RPS8,1:44779125:+,NULL
5,chr1,44778575,44779128,+,ENSG00000142937.12_2,RPS8,1:44779125:+,NULL
6,chr1,42775208,42781977,+,ENSG00000164008.17_2,C1orf50,1:42781980:+,NULL
7,chr1,42775208,42781983,+,ENSG00000164008.17_2,C1orf50,1:42781980:+,NULL
8,chr1,173606034,173610215,-,ENSG00000183831.7_3,ANKRD45,1:173606033:-,NULL
9,chr1,173606030,173610215,-,ENSG00000183831.7_3,ANKRD45,1:173606033:-,NULL


In [37]:
# look at duplicated proximal PAS
sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids) & df.cryptic_status.ne("NULL")).drop_duplicate_positions()[["le_id", "gene_name", "cryptic_status"]]

,Chromosome,Start,End,Strand,le_id,gene_name,cryptic_status
0,chr2,27040954,27041694,+,ENSG00000119777.20_4,TMEM214,background
1,chr2,27041243,27041694,+,ENSG00000119777.20_4,TMEM214,background
2,chr3,52414727,52418329,+,ENSG00000010318.22_1,NaN,background
3,chr3,52415894,52418329,+,ENSG00000010318.22_1,NaN,background
4,chr7,5300662,5307203,+,ENSG00000164638.11_1,NaN,background
5,chr7,5302826,5307203,+,ENSG00000164638.11_1,NaN,background
6,chr7,5303215,5307203,+,ENSG00000164638.11_1,NaN,background
7,chr7,124822385,124823796,-,ENSG00000128513.16_3,POT1,cryptic
8,chr7,124822385,124822551,-,ENSG00000128513.16_3,POT1,cryptic
9,chr11,77322016,77323360,-,ENSG00000149269.10_4,PAK1,background


In [38]:
# All duplicates are equidistant to the atlas site. For the sake of simplicity and consistency arbitrarily select the most distal site for each le_id
# Distal - select most distal/3' PAS for each le_id
# Proximal - like ALEs, select the most proximal/3' site (i.e. shortest annotated exon 5'end)

sel_le_d3utr_prox = select_rep_five_end(sel_le_d3utr_prox, "le_id")
novel_le_rep_d3utr = select_rep_prox_site(novel_le_rep_d3utr, "le_id_quant")


print("Number of proximal PAS regions with distinct 5'ends")
print(sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )


print("Number of distal PAS regions with distinct 3'ends")
print(novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

Number of proximal PAS regions with distinct 5'ends
0
Number of distal PAS regions with distinct 3'ends
0


In [39]:
# Repeat duplicate checks for proximal 3'utr cryptics, first focusing on the novel extensions
print(novel_le_rep_prox_d3utr.cryptic_status.value_counts())
print("\nNumber of duplicated IDs")
# how many IDs are duplicated with distinct regions?
novel_prox_d3utr_dup_ids = set(novel_le_rep_prox_d3utr.drop_duplicate_positions().as_df().le_id_quant.value_counts().loc[lambda x: x > 1].index)
# subset is for le_ids that have more than one entry. Then collapse to a single entry for each le_id and count the number of unique IDs
novel_le_rep_prox_d3utr[novel_le_rep_prox_d3utr.le_id_quant.isin(novel_prox_d3utr_dup_ids)].drop_duplicate_positions().as_df().drop_duplicates(subset="le_id_quant").cryptic_status.value_counts()

cryptic_status
NULL          2494
background     826
cryptic         21
Name: count, dtype: int64

Number of duplicated IDs


cryptic_status
NULL          19
background     9
cryptic        1
Name: count, dtype: int64

In [40]:
# how many events are duplicated when selecting a representative proximal PAS?
sel_prox_d3utr_prox_dup_ids = set(sel_le_prox_d3utr_prox.drop_duplicate_positions().le_id.value_counts().loc[lambda x: x > 1].index)
sel_le_prox_d3utr_prox[sel_le_prox_d3utr_prox.le_id.isin(sel_prox_d3utr_prox_dup_ids)].drop_duplicate_positions().as_df().drop_duplicates(subset="le_id").cryptic_status.value_counts()

cryptic_status
NULL          26
background     7
Name: count, dtype: int64

In [41]:
# no cryptics have duplicated prox sites/distinct 5'ends
# so just need to check 3'end of distal extension
print("Number of distal PAS regions with distinct 3'ends")
print(novel_le_rep_prox_d3utr.subset(lambda df: df.le_id_quant.isin(novel_prox_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

# look at duplicated extensions/distal PAS
# again expect equidistant from atlas site
novel_le_rep_prox_d3utr.subset(lambda df: df.le_id_quant.isin(novel_prox_d3utr_dup_ids) & df.cryptic_status.ne("NULL")).drop_duplicate_positions()[["le_id_quant", "gene_name_ref", "Name", "cryptic_status"]]

Number of distal PAS regions with distinct 3'ends
28


,Chromosome,Start,End,Strand,le_id_quant,gene_name_ref,Name,cryptic_status
0,chr1,70439100,70445530,+,ENSG00000116761.12_2,CTH,1:70445533:+,background
1,chr1,70439100,70445536,+,ENSG00000116761.12_2,CTH,1:70445533:+,background
2,chr1,33494217,33501611,+,ENSG00000121903.15_4,ZSCAN20,1:33501609:+,background
3,chr1,33494217,33501607,+,ENSG00000121903.15_4,ZSCAN20,1:33501609:+,background
4,chr3,133599861,133601393,-,ENSG00000163781.14_6,TOPBP1,3:133599861:-,background
5,chr3,133599859,133601393,-,ENSG00000163781.14_6,TOPBP1,3:133599861:-,background
6,chr6,167969781,167972023,+,ENSG00000130396.21_7,AFDN,6:167972022:+,background
7,chr6,167969781,167972021,+,ENSG00000130396.21_7,AFDN,6:167972022:+,background
8,chr6,111597837,111602289,+,ENSG00000231889.8_6,TRAF3IP2-AS1,6:111602292:+,background
9,chr6,111597837,111602295,+,ENSG00000231889.8_6,TRAF3IP2-AS1,6:111602292:+,background


In [42]:
# like previously, arbitrarily select the most distal site of the duplicated extensions
sel_le_prox_d3utr_prox = select_rep_five_end(sel_le_prox_d3utr_prox, "le_id")
novel_le_rep_prox_d3utr = select_rep_prox_site(novel_le_rep_prox_d3utr, "le_id_quant")

print("Number of proximal PAS regions with distinct 5'ends ")
print(sel_le_prox_d3utr_prox.subset(lambda df: df.le_id.isin(sel_prox_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

print("Number of distal PAS regions with distinct 3'ends")
print(novel_le_rep_prox_d3utr.subset(lambda df: df.le_id_quant.isin(novel_prox_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

Number of proximal PAS regions with distinct 5'ends 
0
Number of distal PAS regions with distinct 3'ends
0


In [43]:
# Construct 'name' string with minimal annotation information
# le_id|gene_name|site_type|cryptic_status

sel_le_d3utr_prox = (sel_le_d3utr_prox
                     .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("ref_gene_name",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("gene_name_common",
                          lambda df: pd.Series(np.where(df["gene_name"].isna(),
                                                        df["ref_gene_name"],
                                                        df["gene_name"])))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_common", "site_type", "cryptic_status"]], sep="|"))
                    
 )


novel_le_rep_d3utr = (novel_le_rep_d3utr
                      .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("Name",
         lambda df: df["le_id_quant"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)


# repeat for proximal cryptics
sel_le_prox_d3utr_prox = (sel_le_prox_d3utr_prox
                     .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("ref_gene_name",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("gene_name_common",
                          lambda df: pd.Series(np.where(df["gene_name"].isna(),
                                                        df["ref_gene_name"],
                                                        df["gene_name"])))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_common", "site_type", "cryptic_status"]], sep="|"))
                    
 )


novel_le_rep_prox_d3utr = (novel_le_rep_prox_d3utr
                      .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("Name",
         lambda df: df["le_id_quant"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)


In [44]:
# quick double check that all cryptic events are retained
print("cryptic 3'UTR extensions - missing cryptic IDs")
print(cryp_le_ids_prtnr - set(sel_le_d3utr_prox.subset(lambda df: df.cryptic_status.eq("cryptic")).le_id))
print(cryp_le_ids - set(novel_le_rep_d3utr.subset(lambda df: df.cryptic_status.eq("cryptic")).le_id_quant))

print("\ncryptic proximal 3'UTRs - missing cryptics")
print(cryp_le_ids_prox - set(sel_le_prox_d3utr_prox.subset(lambda df: df.cryptic_status.eq("cryptic")).le_id))
print(cryp_le_ids_prox_prtnr - set(novel_le_rep_prox_d3utr.subset(lambda df: df.cryptic_status.eq("cryptic")).le_id_quant))

cryptic 3'UTR extensions - missing cryptic IDs
set()
set()

cryptic proximal 3'UTRs - missing cryptics
set()
set()


In [45]:
# generate output bed file of proximal and distal PAS
utr3_out_bed = pr.concat([sel_le_d3utr_prox[["Score", "Name"]],
                          novel_le_rep_d3utr[["Score", "Name"]]]
                          )

utr3_prox_out_bed = pr.concat([sel_le_prox_d3utr_prox[["Score", "Name"]],
                          novel_le_rep_prox_d3utr[["Score", "Name"]]]
                          )

print(f"distal cryptics, number of intervals - {len(utr3_out_bed)}")
# drop duplicate entries
utr3_out_bed = utr3_out_bed.drop_duplicate_positions().sort()

print(f"distal cryptics, number of intervals post duplicate removal - {len(utr3_out_bed)}")
# repeat for proximal cryptics
print(f"proximal cryptics, number of intervals - {len(utr3_prox_out_bed)}")
utr3_prox_out_bed = utr3_prox_out_bed.drop_duplicate_positions().sort()
print(f"proximal cryptics, number of intervals post duplicate removal - {len(utr3_prox_out_bed)}")

distal cryptics, number of intervals - 2014
distal cryptics, number of intervals post duplicate removal - 1770
proximal cryptics, number of intervals - 1888
proximal cryptics, number of intervals post duplicate removal - 1674


In [46]:
# get event counts for cryptic/bg for each region set
# event_counts = {}
# for nm, gr in {"spliced_all_background": spliced_out_bed,
#                "spliced_shsy5y_background": spliced_out_bed_shsy5y_all,
#                "bleedthrough_all_background": bleed_out_bed,
#                "bleed_out_bed_shsy5y_all": bleed_out_bed_shsy5y_all}.items():
    
#     event_counts[nm] = gr.Name.str.split("|", expand=True)[3].rename("reg_status").value_counts()

event_counts = {region: utr3_out_bed.subset(lambda df: df.Name.str.contains(region, regex=False)).Name.str.split("|", expand=True)[3].rename("reg_status").value_counts() 
                for region in ["proximal", "distal"]}

prox_event_counts = {region: utr3_prox_out_bed.subset(lambda df: df.Name.str.contains(region, regex=False)).Name.str.split("|", expand=True)[3].rename("reg_status").value_counts() 
                for region in ["proximal", "distal"]}


event_counts_df = pd.concat(event_counts, names=["event_type"]).reset_index()
prox_event_counts_df = pd.concat(prox_event_counts, names=["event_type"]).reset_index()
print("proximal cryptics event counts")
print(prox_event_counts_df)
print("distal cryptics event count")
event_counts_df

proximal cryptics event counts
  event_type  reg_status  count
0   proximal  background    817
1   proximal     cryptic     20
2     distal  background    817
3     distal     cryptic     20
distal cryptics event count


,event_type,reg_status,count
0,proximal,background,799
1,proximal,cryptic,86
2,distal,background,799
3,distal,cryptic,86


In [47]:
# what are the 19 different genes?
prox_out_nm = set(utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("background", regex=False)).subset(lambda df: df.Name.str.contains("proximal", regex=False)).Name)
out_nm = set(utr3_out_bed.subset(lambda df: df.Name.str.contains("background", regex=False)).subset(lambda df: df.Name.str.contains("proximal", regex=False)).Name)
print(len(prox_out_nm - out_nm))
prox_out_nm - out_nm

19


{'ENSG00000026652.15_2|AGPAT4|proximal|background',
 'ENSG00000065457.11_1|ADAT1|proximal|background',
 'ENSG00000068784.13_1|SRBD1|proximal|background',
 'ENSG00000099381.19_2|SETD1A|proximal|background',
 'ENSG00000105926.16_1|PALS2|proximal|background',
 'ENSG00000105948.13_3|TTC26|proximal|background',
 'ENSG00000108599.15_3|AKAP10|proximal|background',
 'ENSG00000131375.10_2|CAPN7|proximal|background',
 'ENSG00000135315.12_1|CEP162|proximal|background',
 'ENSG00000138138.14_1|ATAD1|proximal|background',
 'ENSG00000138802.11_1|SEC24B|proximal|background',
 'ENSG00000157500.12_3|APPL1|proximal|background',
 'ENSG00000196689.13_1|TRPV1|proximal|background',
 'ENSG00000214562.15_1|NUTM2D|proximal|background',
 'ENSG00000235173.7_1|HGH1|proximal|background',
 'ENSG00000244115.1_1|DNAJC25-GNG10|proximal|background',
 'ENSG00000253741.3_1|LNCOC1|proximal|background',
 'ENSG00000263069.6_2|RNF213-AS1|proximal|background',
 'ENSG00000265808.4_2|SEC22B|proximal|background'}

In [48]:
# any background IDs only found in out BED
out_only = out_nm - prox_out_nm
out_only

{'ENSG00000184441.4_1|ENSG00000184441|proximal|background'}

In [49]:
# is the ID found in the proximal cryptics BED?
utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("|".join([le_id.split("|")[0] for le_id in (out_only)]))).Name.values

array(['ENSG00000184441.4_1|ENSG00000184441|proximal|cryptic'],
      dtype=object)

In [50]:
# is the distal ID of this gene also in the gr and annotated as cryptic?
tmp_id = ["_".join([id, str(int(num) + 1)]) for nm in out_only for id, num in [nm.split("|")[0].split("_")]]
tmp_id
utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("|".join(tmp_id)))

,Chromosome,Start,End,Score,Strand,Name
6,chr21,44333201,44337744,.,+,ENSG00000184441.4_2|ENSG00000184441|distal|cry...


In [51]:
# "|".join([le_id.split("|")[1] for le_id in (prox_out_nm - out_nm)])
# are the 19 missing in the distal cryptics BED
utr3_out_bed.subset(lambda df: df.Name.str.contains("|".join([le_id.split("|")[0] for le_id in (prox_out_nm - out_nm)])))

,Chromosome,Start,End,Score,Strand,Name
0,chr1,120150897,120157192,.,-,ENSG00000265808.4_2|SEC22B|proximal|cryptic
1,chr2,45388679,45389599,.,-,ENSG00000068784.13_1|SRBD1|proximal|cryptic
2,chr3,15251115,15252916,.,+,ENSG00000131375.10_2|CAPN7|proximal|cryptic
3,chr3,57269540,57273471,.,+,ENSG00000157500.12_3|APPL1|proximal|cryptic
4,chr4,109539560,109540896,.,+,ENSG00000138802.11_1|SEC24B|proximal|cryptic
5,chr6,84124240,84125276,.,-,ENSG00000135315.12_1|CEP162|proximal|cryptic
6,chr6,161129966,161136634,.,-,ENSG00000026652.15_2|AGPAT4|proximal|cryptic
7,chr7,24687437,24694193,.,+,ENSG00000105926.16_1|PALS2|proximal|cryptic
8,chr7,139189320,139191986,.,+,ENSG00000105948.13_3|TTC26|proximal|cryptic
9,chr8,144139387,144140851,.,+,ENSG00000235173.7_1|HGH1|proximal|cryptic


In [52]:
# 17 are annotated as cryptic in the distal 3'UTR BED (but background in the proximal BED) - solution is just to remove those genes
# 1 is only found only in the distal bed (cryptic in proximal BED) - should be removed from background
extra_ids_prox = [le_id.split("|")[0] for le_id in (prox_out_nm - out_nm)]
# construct the distal ID for these events
extra_ids_dist = ["_".join([id, str(int(num) + 1)]) for le_id in extra_ids_prox for id, num in [le_id.split("_")]]

# and for IDs only found in distal cryptic BED
out_extra_ids_prox = [le_id.split("|")[0] for le_id in out_only]
out_extra_ids_dist = ["_".join([id, str(int(num) + 1)]) for le_id in out_extra_ids_prox for id, num in [le_id.split("_")]]

utr3_prox_out_bed = utr3_prox_out_bed.subset(lambda df: ~df.Name.str.contains("|".join(set(extra_ids_prox).union(set(extra_ids_dist)))))
utr3_out_bed = utr3_out_bed.subset(lambda df: ~df.Name.str.contains("|".join(set(out_extra_ids_prox).union(set(out_extra_ids_dist)))))
# utr3_prox_out_bed = utr3_prox_out_bed.subset(lambda df: ~df.Name.str.contains("|".join([nm.split("|")[0] for nm in (prox_out_nm - out_nm)])))

prox_event_counts = {region: utr3_prox_out_bed.subset(lambda df: df.Name.str.contains(region, regex=False)).Name.str.split("|", expand=True)[3].rename("reg_status").value_counts() 
                for region in ["proximal", "distal"]}

event_counts = {region: utr3_out_bed.subset(lambda df: df.Name.str.contains(region, regex=False)).Name.str.split("|", expand=True)[3].rename("reg_status").value_counts() 
                for region in ["proximal", "distal"]}

event_counts_df = pd.concat(event_counts, names=["event_type"]).reset_index()
prox_event_counts_df = pd.concat(prox_event_counts, names=["event_type"]).reset_index()

print(prox_event_counts_df)
#
print(event_counts_df)

  event_type  reg_status  count
0   proximal  background    798
1   proximal     cryptic     20
2     distal  background    798
3     distal     cryptic     20
  event_type  reg_status  count
0   proximal  background    798
1   proximal     cryptic     86
2     distal  background    798
3     distal     cryptic     86


In [53]:
# double check for further cryptic leakage?
# does the main 3'UTR BED have any proximal extension cryptics?
tmp_gr = utr3_out_bed.assign("le_id", lambda df: df.Name.str.split("|", expand=True)[0]).assign("cryptic_status", lambda df: df.Name.str.split("|", expand=True)[3])
print("Checking distal cryptics for any proximal cryptics")
print(tmp_gr.subset(lambda df: df.le_id.isin(event_type_le_ids["proximal_3utr_extension"])))

print("Checking proximal cryptics for any distal cryptics")
tmp_gr = utr3_prox_out_bed.assign("le_id", lambda df: df.Name.str.split("|", expand=True)[0]).assign("cryptic_status", lambda df: df.Name.str.split("|", expand=True)[3])
print(tmp_gr.subset(lambda df: df.le_id.isin(event_type_le_ids["distal_3utr_extension"])))
# 

Checking distal cryptics for any proximal cryptics
Empty PyRanges
Checking proximal cryptics for any distal cryptics
Empty PyRanges


In [54]:
if not os.path.exists("../processed/iclip_regions"):
    os.makedirs("../processed/iclip_regions")
    
utr3_out_bed.to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.last_exons.bed")
# proximal 3'UTRs
utr3_out_bed.subset(lambda df: df.Name.str.contains("proximal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.last_exons.proximal.bed")
# distal 3'UTRs
utr3_out_bed.subset(lambda df: df.Name.str.contains("distal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.last_exons.distal.bed")

# also make one with just polYA sites
utr3_out_bed.three_end().to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.pas.bed")
# just proximal
utr3_out_bed.three_end().subset(lambda df: df.Name.str.contains("proximal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.pas.proximal.bed")
# just distal
utr3_out_bed.three_end().subset(lambda df: df.Name.str.contains("distal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.pas.distal.bed")

# repeat for proximal cryptics
utr3_prox_out_bed.to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.last_exons.bed")
# proximal 3'UTRs
utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("proximal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.last_exons.proximal.bed")
# distal 3'UTRs
utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("distal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.last_exons.distal.bed")

# also make one with just polYA sites
utr3_prox_out_bed.three_end().to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.pas.bed")
# just proximal
utr3_prox_out_bed.three_end().subset(lambda df: df.Name.str.contains("proximal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.pas.proximal.bed")
# just distal
utr3_prox_out_bed.three_end().subset(lambda df: df.Name.str.contains("distal", regex=False)).to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.pas.distal.bed")



event_counts_df.to_csv("../processed/iclip_regions/2023-12-15_papa_d3utr.event_counts.tsv", sep="\t", header=True,index=False)

In [55]:
# also make a BED with distal bins
utr3_out_bed_prox = utr3_out_bed.subset(lambda df: df.Name.str.contains("proximal", regex=False))
utr3_out_bed_dist = utr3_out_bed.subset(lambda df: df.Name.str.contains("distal", regex=False))

utr3_prox_out_bed_prox = utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("proximal", regex=False))
utr3_prox_out_bed_dist = utr3_prox_out_bed.subset(lambda df: df.Name.str.contains("distal", regex=False))

# print(utr3_out_bed_dist)
# uniq region of distal 3'UTR
utr3_out_bed_dist_bin = utr3_out_bed_dist.subtract(utr3_out_bed_prox, strandedness="same")
utr3_prox_out_bed_dist_bin = utr3_prox_out_bed_dist.subtract(utr3_prox_out_bed_prox, strandedness="same")
print(utr3_out_bed_dist_bin)
utr3_prox_out_bed_dist_bin

+--------------+-----------+-----------+------------+--------------+-------+
| Chromosome   | Start     | End       | Score      | Strand       | +1    |
| (category)   | (int64)   | (int64)   | (object)   | (category)   | ...   |
|--------------+-----------+-----------+------------+--------------+-------|
| chr1         | 6220805   | 6221220   | .          | +            | ...   |
| chr1         | 11296049  | 11299544  | .          | +            | ...   |
| chr1         | 15940456  | 15940477  | .          | +            | ...   |
| chr1         | 19260128  | 19264037  | .          | +            | ...   |
| ...          | ...       | ...       | ...        | ...          | ...   |
| chrX         | 117894639 | 117897812 | .          | -            | ...   |
| chrX         | 119584579 | 119588336 | .          | -            | ...   |
| chrX         | 154505228 | 154506158 | .          | -            | ...   |
| chrX         | 155478419 | 155489010 | .          | -            | ...   |

,Chromosome,Start,End,Score,Strand,Name
0,chr1,6220805,6221220,.,+,ENSG00000158286.13_4|RNF207|distal|background
1,chr1,11296049,11299544,.,+,ENSG00000120942.14_3|UBIAD1|distal|background
2,chr1,19260128,19264037,.,+,ENSG00000053372.5_3|MRTO4|distal|background
3,chr1,24472976,24476667,.,+,ENSG00000001461.17_6|NIPAL3|distal|background
4,chr1,27306185,27307443,.,+,ENSG00000142784.16_2|WDTC1|distal|background
...,...,...,...,...,...,...
857,chrX,110673482,110673855,.,-,ENSG00000101938.15_2|CHRDL1|distal|background
858,chrX,117894639,117897812,.,-,ENSG00000003096.14_2|KLHL13|distal|background
859,chrX,119584579,119588336,.,-,ENSG00000186416.18_2|NKRF|distal|background
860,chrX,154505228,154506158,.,-,ENSG00000071889.17_3|FAM3A|distal|background


In [56]:
utr3_out_bed_dist_bin.to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr.background_shsy5y.last_exon_bins.distal.bed")
utr3_prox_out_bed_dist_bin.to_bed("../processed/iclip_regions/2023-12-15_papa_cryptic_d3utr_proximal.background_shsy5y.last_exon_bins.distal.bed")